In [15]:
import os
import json
from litex import RemoteClient

from functools import reduce

In [16]:
soc_json = "../soc/streamliner.json"

In [17]:
bus = RemoteClient()
bus.open()

In [18]:
with open(soc_json) as file:
  soc = json.load(file)

In [19]:
def set_base(addr):
    bus.write(soc['csr_registers']['wb_udp_tx_dma_base']['addr'], addr)

def read_base():
    return bus.read(soc['csr_registers']['wb_udp_tx_dma_base']['addr'])

def set_length(len):
    bus.write(soc['csr_registers']['wb_udp_tx_dma_length']['addr'], len)

def set_enable(dat):
    bus.write(soc['csr_registers']['wb_udp_tx_dma_enable']['addr'], dat)

def read_done():
    return bus.read(soc['csr_registers']['wb_udp_tx_dma_done']['addr'])

def set_loop(dat):
    bus.write(soc['csr_registers']['wb_udp_tx_dma_loop']['addr'], dat)

def set_port(src_port, dst_port):
    bus.write(soc['csr_registers']['wb_udp_tx_dma_srcdst_port']['addr'], (src_port<<16) + dst_port)

def str_ip4_to_num(x:str) -> int:
    return reduce(lambda x,y: x|y, map(lambda ix: (int(ix[1]) << (8 * ix[0])), enumerate(reversed(x.split(".")))))

def set_ip(ip:str):
    bus.write(soc['csr_registers']['wb_udp_tx_dma_dst_ip']['addr'], str_ip4_to_num(ip))

In [20]:
set_base(0x40000004)

In [21]:
hex(read_base())

'0x40000004'

In [22]:
set_base(0x40000000)
set_port(5123, 5123)
set_ip("192.168.100.20")
set_loop(0)
set_enable(0xffffff)

In [11]:
read_done()

0

In [23]:
bus.close()

In [ ]:
#!/usr/bin/env python3

bus = RemoteClient()
bus.open()

# # #

# Access SDRAM
bus.write(0x40000000, 0x12345678)
value = bus.read(0x40000000)

# Access SDRAM (with bus.mems and base address)
bus.write(bus.mems.main_ram.base, 0x12345678)
value = bus.read(bus.mems.main_ram.base)

# Trigger a reset of the SoC
bus.regs.ctrl_reset.write(1)
 
# Dump all CSR registers of the SoC
for name, reg in bus.regs.__dict__.items():
    print("0x{:08x} : 0x{:08x} {}".format(reg.addr, reg.read(), name))

[...]

# # #

bus.close()
